# Lesson 5

time: 30m

## Learning outcomes:

- introduction to classes in Python
- "dunder" methods
- private and public methods and variables
- saving and loading objects
- caching

We now want to see how good the methods we have developed for computing the Riemann zeta function are.
So far we have mainly been working in Python using **functional** or **imperative** programming styles. An alternative is so-called **object-oriented** programming where the basic building blocks are **objects** belonging to **classes** (think of elements belonging to sets or vectors in vector spaces).
A class is simply a collection of properties that a certain type of objects share. 

One of the good things with Python is that it is easy to use a combination of these styles and simply adapt to what is most natural at the moment. 

Most of SageMath is implemented in an object oriented manner, where **classes** and **subclasses** often correspond to categories and subcategories or spaces and subspaces etc. in a natural way.

One reason for using OO programming is simply to organise your code. 
Let's consider the following class which can help us to organise the code we have written so far. 



In [ ]:
class ZetaNumerical(SageObject):
    r"""
    Class to compute the Riemann zeta function.
    """
    def __init__(self,prec=53, maxn=100): #  'dunder' init function
        """
        Initialize self.
        """
        # define some "private" properties
        self._prec = prec
        self._base_ring = ComplexField(prec)
        self._maxn = maxn

    # convenience methods for accessing (not modifying) private variables
    def prec(self):
        r"""
        Return precision of self.
        """
        return self._prec
    
    def base_ring(self):
        r"""
        Return the base ring of self.
        """
        return self._base_ring
    
    
    def __call__(self,s): ##  'dunder' call function used when class element is "called"
        #
        # The 'r' in front of """ means that this is a "raw" string, otherwise there 
        # would have been an error from the '\z' here. 
        r"""
        Evaluate \zeta(s)
        
        INPUT:
        
        - ``s`` -- complex number
        
        
        EXAMPLE:
        
            sage: z = ZetaNumerical(103)
            sage: z(10)
            ...
            sage: z(x)
            ...
            sage: Z=ZetaNumerical(prec=53,maxn=100)
            sage: x=delta_lseries(10)
            sage: Z(x)
            ValueError                             
            ...
            ValueError: Could not coerce L-series of conductor 1 and weight 12 to a complex number!
        """
        try: 
            scplx = self._base_ring(s)
        except TypeError: # Type error is raised when an internal function can not apply on this type
            raise ValueError(f"Could not coerce {s} to a complex number!")
        if scplx.real() > 1:
            return self._sum(scplx)
        return self._euler_mclaurin(scplx)
    
    # Private method starting with '_'. 
    # Should not be called explicitly from outside this class
    def _sum(self,s):
        r"""
        Evaluate self using a "naive" sum.
        """
        return sum([self._base_ring(n)**-s for n in range(1,self._maxn)])
    
    def _euler_mclaurin(self,s):
        r"""
        Evaluate self using the Euler McLaurin summation formula.
        """
        # When writing a class it is useful to populate it with methods you would like
        # and then simply mark these as 
        raise NotImplementedError("This method has not been implemented yet!")
        
        

In [ ]:
Z=ZetaNumerical(prec=53,maxn=100)
Z

**Exercise**
Add the following two methods to the class above:
1. A "dunder method" called `__repr__` which determines how the object is displayed
2. A method to plot values of zeta along a vertical line. Input should at minimum be the x coordinate, the endpoints of the line and the number of points. 
Hint: you might want to look at the function `parametric_plot`.

Since we inherited from SageObject we can easily save and load objects

In [ ]:
# Saves the object in a binary file 'my_object.sobj'
Z.save('my_object.sobj')

In [ ]:
# Loads the object from file
Z1 = load('my_object.sobj')

In [ ]:
# Do we get back the same object? 
Z1 == Z

The reason this doesn't work is that our class `ZetaNumerical` does not know **how** to compare anything.
We can check that the data given by the loaded object is the same as that of the original:

In [ ]:
Z.__dict__

In [ ]:
Z1.__dict__

**Exercise**
Add another dunder method `__eq__` which takes as input `self` and `other` returns True if other is an object of the same type as self and has the same precision and maxn parameters, and otherwise returns False. 


It is possible to use methods from other python files by either using 
- `load`
- `attach`
However, to import notebooks is a bit more difficult. 
One option is to use the magic `%run` command and a full path name of the file.
Note that this will run the entire notebook...

In [ ]:
%run ExampleSolutions.ipynb

**Exercise**
Make the call method of `ZetaNumerical` complete by either using the Euler-McLaurin method you implemented earlier (or the one from the "solutions"), or if you want to be quicker, cheat and use the builtin `zeta`.


## Caching in Sage

The `@cached_function` and `@cached_method` decorators can be used to store computed values and avoid recomputing the same thing twice


In [ ]:
class MyClass(SageObject):
    
    def computation(self,x):
        print("doing computation")
        return CC(sum([n**-x for n in range(1,1000)]))

    @cached_method   
    def computation_cached(self,x):
        print("doing another computation")
        return CC(sum([n**-x for n in range(1,1000)]))

    

In [ ]:
X=MyClass()

In [ ]:
X.computation(2)

In [ ]:
X.computation_cached(2)

In [ ]:
X.computation(2)

In [ ]:
X.computation_cached(3)

**Exercise**
Add caching to relevant methods in your class. 